# Content List
### 01. Importing libraries
### 02. Importing data
### 03. Step 4: Wrangling new customer data
### 04. Step 5: Data Quality and Consistency Checks
### 05. Step 6: Combine customers data with orders_products data
### 06. Step 8: Export new combined dataframe

# 01. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Importing data

In [2]:
# create path to main project folder
path = r'/Users/kirstinhelgeson/Desktop/Data Analytics Course/05-2023 Instacart Basket Analysis'

In [3]:
# import customers.csv data set
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
# import orders_products_aggregated.pkl dataframe
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))

# 03. Step 4: Wrangling new customer data

In [4]:
# check columns listed in customers.csv dataframe for unnecessary or incorrect names
df_customers.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [5]:
# check dataframe df_customers size
df_customers.shape

(206209, 10)

In [9]:
# check basic information of df_customers
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


### First Name has 194950 non-null entries. This means we have 206209 - 194950 = 11259 null values. Confirmed below:

In [11]:
# double checking missing values in 'First Name' column
df_customers['First Name'].value_counts(dropna = False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

### There are 11259 missing first name values, about 5.5% of all First Name data. Without outside sources we cannot replace or remove these values. They should not affect any statistical values so we can leave them blank for now.

In [12]:
# check descriptive statistics for df_customers
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [13]:
# check output of df_customers, first 10 rows
df_customers.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [15]:
# rename "Surnam" to "Last Name" to better fit the format given by "First Name" column in df_customers
df_customers.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)

In [17]:
# rename "STATE" to "State" to fit format of other column names in df_customers
df_customers.rename(columns = {'STATE' : 'State'}, inplace = True)

In [18]:
# confirm changes to column names by checking output
df_customers.head()

,user_id,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# 04. Step 5: Data Quality and Consistency Checks

In [19]:
# check for mixed-type data in df_customers
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_customers[weird]) > 0:
        print(col)

First Name


### First Name has mixed-type data because there are null values. The other columns are not mixed-type. Change the data type to String to fix this.

In [20]:
# change data type of "First Name" to a String
df_customers['First Name'] = df_customers['First Name'].astype('str')

In [21]:
# check for mixed-types again to confirm change
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_customers[weird]) > 0:
        print(col)

In [22]:
# check for missing values. 'First Name' column has missing values but data type has been changed to String
df_customers.isnull().sum()

user_id         0
First Name      0
Last Name       0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [23]:
# find any duplicate entries in df_customers and assign to subset df_dups
df_dups = df_customers[df_customers.duplicated()]

In [24]:
# print df_dups subset
df_dups

,user_id,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income


### There are no duplicates in the df_customers dataframe.

# 05. Step 6: Combine customers data with orders_products data

In [25]:
# check df_ords_prods information
df_ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  busiest_days            object 
 15  busiest_period_of_day   object 
 16  max_order               int64  
 17  loyalty_flag            object 
 18  mean_price              float64
 19  spending_flag           object 
 20  median_days             float64
 21  frequency_flag          objec

In [26]:
# check df_ords_prods dataframe size
df_ords_prods.shape

(32404859, 22)

In [27]:
# checking df_customers info again
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    206209 non-null  object
 2   Last Name     206209 non-null  object
 3   Gender        206209 non-null  object
 4   State         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


### "user_id" will be the key used to merge these two dataframes

In [28]:
# combine df_customers with df_ords_prods using the 'user_id' column
df_combined = df_ords_prods.merge(df_customers, on = 'user_id')

In [29]:
# check first few rows of df_combined dataframe
df_combined.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [30]:
# check size of df_combined dataframe
df_combined.shape

(32404859, 31)

In [31]:
# checking basic info of df_combined dataframe to confirm correct info
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 31 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  busiest_days            object 
 15  busiest_period_of_day   object 
 16  max_order               int64  
 17  loyalty_flag            object 
 18  mean_price              float64
 19  spending_flag           object 
 20  median_days             float64
 21  frequency_flag          objec

# 06. Step 8: Export new combined dataframe

In [32]:
#Exporting combined df_customer data set with df_ords_prods to pickle file
df_combined.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customers_ords_prods_merged.pkl'))